**<small>Abstract:</small>**

<small>This genomic analysis involves a series of bioinformatics steps to process and assess the quality of sequencing data. Initially, raw genomic data files (FASTA or FASTQ) are downloaded and decompressed to obtain readable sequences. Subsets of these sequences are extracted for efficient processing using commands such as head. The reference genome is indexed with the Burrows-Wheeler Aligner (BWA) tool, which prepares the genome for fast and accurate alignment. Subsequently, sequencing reads are aligned to the indexed reference genome using the BWA-MEM algorithm, producing alignment files in SAM format. Quality control of the sequencing reads is conducted using FastQC, which generates detailed reports and visualizations to evaluate read quality, sequence length distribution, GC content, and other important metrics. </small>

**<small>Install Miniconda</small>**

In [1]:
%%bash
# Download the Miniconda installer for Python 3.9 (Linux 64-bit), suppressing output
wget https://repo.anaconda.com/miniconda/Miniconda3-py39_4.10.3-Linux-x86_64.sh &>/dev/null

# Make the installer executable
chmod u+x Miniconda3-py39_4.10.3-Linux-x86_64.sh

# Run the installer in batch mode, force overwrite if needed, and install to /usr/local (no output)
./Miniconda3-py39_4.10.3-Linux-x86_64.sh -b -f -p /usr/local &>/dev/null

# Delete the installer script after installation
rm Miniconda3-py39_4.10.3-Linux-x86_64.sh

**<small>Install the package manager Mamba</small>**

In [2]:
%%bash
# Install the 'mamba' package manager from the conda-forge channel without prompting for confirmation
conda install -c conda-forge mamba --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mamba


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       h4bc722e_7         247 KB  conda-forge
    c-ares-1.34.5              |       hb9d3cd8_0         202 KB  conda-forge
    ca-certificates-2025.6.15  |       hbd8a1cb_0         148 KB  conda-forge
    certifi-2025.6.15          |     pyhd8ed1ab_0         152 KB  conda-forge
    cffi-1.17.1                |   py39h15c3d72_0         236 KB  conda-forge
    conda-22.11.1              |   py39hf3d152e_1         904 KB  conda-forge
    cpp-expected-1.1.0         |       hff21bea_1          24 KB  conda-forge
    cryptography-45.0.4        |   py39h7170ec2_0         1.5 MB  conda-forge
    fmt-11.1.4                 |       h07f6e7f_1         187 KB  conda-forge
    krb5-1.21.3   



==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda




In [3]:
%%bash
# Download the latest version of micromamba for Linux 64-bit and extract only the binary file
wget -qO- https://micromamba.snakepit.net/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

# Create the target directory for micromamba if it doesn't exist
mkdir -p /usr/local/micromamba

# Move the micromamba binary to the target directory
mv bin/micromamba /usr/local/micromamba/

# Remove the now-empty 'bin' directory after moving the binary
rm -r bin

bin/micromamba


In [4]:
%%bash
# Create a new micromamba environment named 'lab11'
# The environment will be created using packages from the bioconda and conda-forge channels
# The only package installed in this command is 'bwa' (a tool for aligning sequencing reads)
# The '-y' flag automatically confirms the installation
/usr/local/micromamba/micromamba create -y -n lab11 -c bioconda -c conda-forge \
   bwa



Transaction

  Prefix: /usr/local/envs/lab11

  Updating specs:

   - bwa


  Package          Version  Build             Channel           Size
──────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────

  + _libgcc_mutex      0.1  conda_forge       conda-forge        3kB
  + _openmp_mutex      4.5  2_gnu             conda-forge       24kB
  + bwa             0.7.19  h577a1d6_1        bioconda         654kB
  + libgcc          15.1.0  h767d61c_3        conda-forge     Cached
  + libgcc-ng       15.1.0  h69a702a_3        conda-forge     Cached
  + libgomp         15.1.0  h767d61c_3        conda-forge     Cached
  + libxcrypt       4.4.36  hd590300_1        conda-forge     Cached
  + libzlib          1.3.1  hb9d3cd8_2        conda-forge     Cached
  + perl            5.32.1  7_hd590300_perl5  conda-forge       13MB

  Summary:

  Install: 9 packages

  Total download: 14MB

──────────────────

In [8]:
%%bash
# Initialize the micromamba shell hook for bash
# This sets up the shell so that micromamba commands (like activate) work properly
eval "$(/usr/local/micromamba/micromamba shell hook --shell=bash)"

# Activate the 'lab11' environment created earlier
micromamba activate lab11

# Run the 'bwa' command to check if the tool was installed correctly
bwa 2>&1 | head -n 10


Program: bwa (alignment via Burrows-Wheeler transformation)
Version: 0.7.19-r1273
Contact: Heng Li <hli@ds.dfci.harvard.edu>

Usage:   bwa <command> [options]

Command: index         index sequences in the FASTA format
         mem           BWA-MEM algorithm
         fastmap       identify super-maximal exact matches


**<small>Genomic Data Analysis</small>**

In [14]:
%%bash
# Download the compressed FASTA file containing structural variant contigs (svtigs)
# for the NA12878 individual from the 1000 Genomes Project (Oxford Nanopore Vienna dataset)
wget https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1KG_ONT_VIENNA/release/v1.0/svarp/svtigs/NA12878/NA12878_svtigs_H1.fa.gz

# Uncompress the .gz file to get the actual FASTA file (NA12878_svtigs_H1.fa)
gunzip /content/NA12878_svtigs_H1.fa.gz

--2025-07-02 10:47:40--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1KG_ONT_VIENNA/release/v1.0/svarp/svtigs/NA12878/NA12878_svtigs_H1.fa.gz
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2772567 (2.6M) [application/x-gzip]
Saving to: ‘NA12878_svtigs_H1.fa.gz’

     0K .......... .......... .......... .......... ..........  1%  184K 14s
    50K .......... .......... .......... .......... ..........  3%  369K 11s
   100K .......... .......... .......... .......... ..........  5%  368K 9s
   150K .......... .......... .......... .......... ..........  7% 63.6M 7s
   200K .......... .......... .......... .......... ..........  9%  370K 7s
   250K .......... .......... .......... .......... .......... 11% 80.1M 5s
   300K .......... .......... .......... .......... .......... 12% 63.6

In [17]:
%%bash
# Initialize the micromamba shell hook for bash
# This sets up the shell so that micromamba commands (like activate) work properly
eval "$(/usr/local/micromamba/micromamba shell hook --shell=bash)"

# Activate the 'lab11' environment created earlier
micromamba activate lab11

# Extract the first 100 lines from the FASTA file and save them into a smaller file named 'test.fasta'
head -n 100 /content/NA12878_svtigs_H1.fa > test.fasta

# Build BWA index files for the reference genome FASTA file 'Homo_sapiens.GRCh38.dna.chromosome.1.fa'
# The generated index files will use the prefix 'human' to identify them
bwa index /content/NA12878_svtigs_H1.fa human

[bwa_index] Pack FASTA... 0.12 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 5.03 seconds elapse.
[bwa_index] Update BWT... 0.06 sec
[bwa_index] Pack forward-only FASTA... 0.05 sec
[bwa_index] Construct SA from BWT and Occ... 2.33 sec
[main] Version: 0.7.19-r1273
[main] CMD: bwa index /content/NA12878_svtigs_H1.fa human
[main] Real time: 7.693 sec; CPU: 7.583 sec


In [18]:
%%bash
# Decompress the file 'NA12878_svtigs_H1.fa' and output its contents
# Then, take the first 40,000 lines from the decompressed data
# Save these lines into a new file named 'test_reads.fastq' for further analysis
zcat /content/NA12878_svtigs_H1.fa | head -n 40000 >test_reads.fastq


gzip: /content/NA12878_svtigs_H1.fa: not in gzip format


In [ ]:
%%bash
# Extract the first 100 lines from the FASTA file and save to 'test1_reference.fasta'
head -n 100 /content/test.fasta >test1_reference.fasta

# Initialize the micromamba shell hook for bash
# This sets up the shell so that micromamba commands (like activate) work properly
eval "$(/usr/local/micromamba/micromamba shell hook --shell=bash)"

# Activate the 'lab11' environment created earlier
micromamba activate lab11

# Decompress the FASTQ file and extract the first 20,000 lines to create a smaller sample 'test1.fastq'
zcat SRR101470_2.fastq.gz | head -n 20000 >test1.fastq

# Build BWA index files for the reference genome 'test1_reference.fasta'
# The index files will be prefixed with 'test1_reference'
bwa index test1_reference.fasta test1_reference

# Align the reads in 'test1.fastq' against the indexed reference genome
# Output the alignment in SAM format to 'test1.sam'
bwa mem test1_reference.fasta test1.fastq >test1.sam

# Display the first 100 lines of the SAM alignment output to verify results
head -100 test1.sam

@HD	VN:1.5	SO:unsorted	GO:query
@SQ	SN:H1-s10005_2483	LN:5940
@PG	ID:bwa	PN:bwa	VN:0.7.19-r1273	CL:bwa mem test1_reference.fasta test1.fastq


gzip: SRR101470_2.fastq.gz: No such file or directory
[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.19-r1273
[main] CMD: bwa index test1_reference.fasta test1_reference
[main] Real time: 0.032 sec; CPU: 0.005 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[main] Version: 0.7.19-r1273
[main] CMD: bwa mem test1_reference.fasta test1.fastq
[main] Real time: 0.011 sec; CPU: 0.003 sec


In [21]:
%%bash
# Initialize the micromamba shell hook for bash
# This sets up the shell so that micromamba commands (like activate) work properly
eval "$(/usr/local/micromamba/micromamba shell hook --shell=bash)"

# Activate the 'lab11' environment created earlier
micromamba activate lab11

# Filter lines from 'test1.sam' where the 9th field (insert size or template length) is not zero
# This can help exclude unmapped or improperly paired reads, focusing on reads with meaningful alignment info
awk '{if ($9 != 0)}' test1.sam

In [27]:
%%bash
# Initialize the micromamba shell hook for bash
# This sets up the shell so that micromamba commands (like activate) work properly
eval "$(/usr/local/micromamba/micromamba shell hook --shell=bash)"

# Install the 'fastqc' package into the 'lab11' environment from the bioconda channel, auto-confirming prompts
micromamba install -n lab11 -c bioconda fastqc --yes



Transaction

  Prefix: /usr/local/envs/lab11

  Updating specs:

   - fastqc


  Package     Version  Build  Channel       Size
──────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────

  + fastqc     0.11.5  1      bioconda      10MB
  + java-jdk   8.0.92  1      bioconda     128MB

  Summary:

  Install: 2 packages

  Total download: 138MB

──────────────────────────────────────────────────



Transaction starting
Linking java-jdk-8.0.92-1
Linking fastqc-0.11.5-1

Transaction finished



In [50]:
%%bash
# Initialize the micromamba shell hook for bash
# This sets up the shell so that micromamba commands (like activate) work properly
eval "$(/usr/local/micromamba/micromamba shell hook --shell=bash)"

# Activate the 'lab11' environment created earlier
micromamba activate lab11

# Run FastQC quality control analysis on the input FASTQ file located at /content/test1.fastq
fastqc /content/test1.fastq

Analysis complete for test1.fastq


Started analysis of test1.fastq
Failed to process file test1.fastq
java.lang.ArrayIndexOutOfBoundsException: -1
	at uk.ac.babraham.FastQC.Modules.SequenceLengthDistribution.calculateDistribution(SequenceLengthDistribution.java:100)
	at uk.ac.babraham.FastQC.Modules.SequenceLengthDistribution.raisesError(SequenceLengthDistribution.java:184)
	at uk.ac.babraham.FastQC.Report.HTMLReportArchive.startDocument(HTMLReportArchive.java:336)
	at uk.ac.babraham.FastQC.Report.HTMLReportArchive.<init>(HTMLReportArchive.java:84)
	at uk.ac.babraham.FastQC.Analysis.OfflineRunner.analysisComplete(OfflineRunner.java:155)
	at uk.ac.babraham.FastQC.Analysis.AnalysisRunner.run(AnalysisRunner.java:110)
	at java.lang.Thread.run(Thread.java:745)
